# CMSC 173 - MP 2
## Instructions:
1. Create an overview of the problem being solved, e.g., what was the story behind the collection of the data, description of the attributes/features used,etc.
2. (Data Preprocessing and Exploratory Analysis) Present descriptive statistics as applicable (e.g., distribution, central tendency, variability) of the data before training the models. Clean the data if there are missing values, etc. You may perform feature engineering (i.e., creating new features out of the given features), but be sure to document your justifications. 
3. Split your data into proportions of 70% training set and 30% testing set.
4. Train the following models: (a) logistic regression classifier and (b) naive Bayes classifier on the dataset.
5. Evaluate the performance of the trained model. You may use additional performance measures if you want, but for now I will only require the calculation of the accuracy. The accuracy measures the fraction of correct classifications. With this, you need to generate the confusion matrix. You may read this if you haven't encountered this concept before: https://www.sciencedirect.com/topics/engineering/confusion-matrix#:~:text=A%20confusion%20matrix%20represents%20the,by%20model%20as%20other%20class. Remember to compute this matrix from the test set (not the training set).

In [236]:
using Random
using StatsBase
using CSV
using DataFrames 
using Plots
using Base
import StatisticalMeasures.ConfusionMatrices as CM

In [237]:
dataset = CSV.read("passenger_flight.csv",DataFrame)
Random.seed!(123)
dataset = dataset[shuffle(axes(dataset, 1)), :]

Row,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64?,Int64
1,1,1,50,1,1,3744,5,5,5,5,3,4,4,4,4,4,4,5,4,3,0,0,1
2,0,1,53,1,1,2661,4,5,5,5,3,1,2,4,4,3,4,4,4,2,6,8,0
3,1,1,20,0,0,541,2,4,2,3,4,2,4,4,2,2,4,2,3,4,38,38,0
4,0,1,52,0,1,944,1,2,1,2,2,3,2,2,2,1,2,1,2,2,34,48,0
5,1,1,33,1,1,406,1,1,1,1,4,4,4,4,3,5,4,3,5,4,0,0,1
6,0,1,51,0,0,621,2,4,2,1,2,4,4,3,3,2,3,5,3,3,0,0,0
7,1,1,25,1,1,3547,2,2,2,2,5,5,5,5,5,5,1,4,4,5,0,0,1
8,0,1,51,1,1,547,4,4,4,4,2,4,5,4,4,4,4,3,4,5,0,0,1
9,0,1,60,0,1,438,2,4,2,3,2,4,4,5,5,2,5,5,5,3,0,0,0


## Data Preprocessing

In [238]:
# Replace missing values with the mean

has_missing = .!completecases(dataset)
rows_with_missing_values = dataset[has_missing, :] 
print(rows_with_missing_values) # 83 rows have missing values in the Arrival Delay in Minutes column

mean_value = mean(skipmissing(dataset[:,"Arrival Delay in Minutes"]))
transform!(dataset, All() .=> (x -> replace(x, missing => mean(skipmissing(x)))) => identity)
println(dataset[1:10,:])

has_missing = .!completecases(dataset)
rows_with_missing_values = dataset[has_missing, :] 
display(rows_with_missing_values) # no missing values

83×23 DataFrame
 Row │ Gender  Customer Type  Age    Type of Travel  Class  Flight Distance  Inflight wifi service  Departure/Arrival time convenient  Ease of Online booking  Gate location  Food and drink  Online boarding  Seat comfort  Inflight entertainment  On-board service  Leg room service  Baggage handling  Checkin service  Inflight service  Cleanliness  Departure Delay in Minutes  Arrival Delay in Minutes  satisfaction 
     │ Int64   Int64          Int64  Int64           Int64  Int64            Int64                  Int64                              Int64                   Int64          Int64           Int64            Int64         Int64                   Int64             Int64             Int64             Int64            Int64             Int64        Int64                       Union{Missing, Int64}     Int64        
─────┼────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Row,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64


In [239]:
# rename column names
col_names = names(dataset)
new_col_names = map(lowercase, String.(col_names)) # convert to lower case
new_col_names .= replace.(new_col_names, " "=>"_", "-"=>"", "/"=>"_") # replace spaces and slash with underscore and remove dashes, 
rename!(dataset, new_col_names)
display(dataset)

Row,gender,customer_type,age,type_of_travel,class,flight_distance,inflight_wifi_service,departure_arrival_time_convenient,ease_of_online_booking,gate_location,food_and_drink,online_boarding,seat_comfort,inflight_entertainment,onboard_service,leg_room_service,baggage_handling,checkin_service,inflight_service,cleanliness,departure_delay_in_minutes,arrival_delay_in_minutes,satisfaction
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1.0,1.0,50.0,1.0,1.0,3744.0,5.0,5.0,5.0,5.0,3.0,4.0,4.0,4.0,4.0,4.0,4.0,5.0,4.0,3.0,0.0,0.0,1.0
2,0.0,1.0,53.0,1.0,1.0,2661.0,4.0,5.0,5.0,5.0,3.0,1.0,2.0,4.0,4.0,3.0,4.0,4.0,4.0,2.0,6.0,8.0,0.0
3,1.0,1.0,20.0,0.0,0.0,541.0,2.0,4.0,2.0,3.0,4.0,2.0,4.0,4.0,2.0,2.0,4.0,2.0,3.0,4.0,38.0,38.0,0.0
4,0.0,1.0,52.0,0.0,1.0,944.0,1.0,2.0,1.0,2.0,2.0,3.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,34.0,48.0,0.0
5,1.0,1.0,33.0,1.0,1.0,406.0,1.0,1.0,1.0,1.0,4.0,4.0,4.0,4.0,3.0,5.0,4.0,3.0,5.0,4.0,0.0,0.0,1.0
6,0.0,1.0,51.0,0.0,0.0,621.0,2.0,4.0,2.0,1.0,2.0,4.0,4.0,3.0,3.0,2.0,3.0,5.0,3.0,3.0,0.0,0.0,0.0
7,1.0,1.0,25.0,1.0,1.0,3547.0,2.0,2.0,2.0,2.0,5.0,5.0,5.0,5.0,5.0,5.0,1.0,4.0,4.0,5.0,0.0,0.0,1.0
8,0.0,1.0,51.0,1.0,1.0,547.0,4.0,4.0,4.0,4.0,2.0,4.0,5.0,4.0,4.0,4.0,4.0,3.0,4.0,5.0,0.0,0.0,1.0
9,0.0,1.0,60.0,0.0,1.0,438.0,2.0,4.0,2.0,3.0,2.0,4.0,4.0,5.0,5.0,2.0,5.0,5.0,5.0,3.0,0.0,0.0,0.0


## Detection of Outliers and Removal

In [240]:
q1 = []
q3 = []

col_names = names(dataset)

for i in col_names
    push!(q1, quantile(dataset[:,i], 0.25))
    push!(q3, quantile(dataset[:,i], 0.75))
end
iqr_val = q3-q1

lower_bound =  q1 - 1.5 * iqr_val
upper_bound = q3 + 1.5 * iqr_val

outlier = BitVector()

for row in eachrow(dataset)
    is_outlier = 0

    for col_idx in 1:length(col_names)
        if row[col_idx] < lower_bound[col_idx] || row[col_idx] > upper_bound[col_idx]
            is_outlier = 1
        end
    end
    push!(outlier, is_outlier)
end

cleaned_dataset = dataset[.!outlier,:]
println("Number of rows before removal: ", size(dataset)[1])
println("Number of rows after removal: ", size(cleaned_dataset)[1])


Number of rows before removal: 25976
Number of rows after removal: 15234


In [241]:
# split dataframe into 2 df depending on pct
function splitdf(df, pct)
    @assert 0 <= pct <= 1
    ids = collect(axes(df, 1))
    shuffle!(ids)
    sel = ids .<= nrow(df) .* pct
    train = view(df, sel, :)
    test = view(df, .!sel, :)

    # println(hcat(train[:,1:end-1], DataFrame("satisfaction"=>train[:,end])) == train)

    return train[:,1:end-1], DataFrame("satisfaction"=>train[:,end]), test[:,1:end-1], DataFrame("satisfaction" => test[:,end])
end

(x_train, y_train, x_test, y_test) = splitdf(cleaned_dataset, 0.7)

(10663×22 DataFrame
   Row │ gender   customer_type  age      type_of_travel  class    flight_dist ⋯
       │ Float64  Float64        Float64  Float64         Float64  Float64     ⋯
───────┼────────────────────────────────────────────────────────────────────────
     1 │     0.0            1.0     53.0             1.0      1.0           26 ⋯
     2 │     1.0            1.0     33.0             1.0      1.0            4
     3 │     0.0            1.0     51.0             0.0      0.0            6
     4 │     1.0            1.0     25.0             1.0      1.0           35
     5 │     0.0            1.0     51.0             1.0      1.0            5 ⋯
     6 │     0.0            1.0     60.0             0.0      1.0            4
     7 │     1.0            1.0     17.0             0.0      0.0            5
     8 │     0.0            1.0     24.0             0.0      0.0            6
     9 │     0.0            1.0     31.0             1.0      1.0           35 ⋯
    10 │     1.0    

## Naive Bayes

In [242]:
# build conditional probability table
cont_col_names = ["age", "flight_distance", "departure_delay_in_minutes", "arrival_delay_in_minutes"]
disc_col_names = [name for name in names(dataset) if name ∉ cont_col_names && name≠"satisfaction"]
train = hcat(x_train, y_train)

# calculate discrete probabilities
function count_disc_prob(df, col_name)
    return combine(groupby(df, [col_name, "satisfaction"]), nrow)
end

cond_prob_table = Dict()
for name in disc_col_names
    cond_prob_table[name] = count_disc_prob(train, name)
end

# calculate continuous probabilities
function count_cont_prob(df, col_name)
    a = combine(groupby(df, "satisfaction"), [col_name] => mean, [col_name] => std)
end

for name in cont_col_names
    cond_prob_table[name] = count_cont_prob(train, name)
end

In [243]:
# calculate likelihood for continuous data
function likelihood(cond_prob_table, feature, satisfaction, x)
    feature_table = cond_prob_table[feature]
    prob_values = filter(row -> row.satisfaction == satisfaction, feature_table)

    # get mean and variance
    μ = prob_values[1,2]
    σ = prob_values[1,3]

    return (1/(σ * sqrt(2π))) * exp((-1/2) * ((x-μ)/σ)^2)
end

# calculate probabilities for discrete (categorical) data
function disc_cond_prob(cond_prob_table, feature, satisfaction, x)
    feature_table = cond_prob_table[feature]
    feature_table = filter(row -> row.satisfaction==satisfaction, feature_table)
    total = sum(feature_table[:,:nrow])
    
    val = 0
    try
        val = filter(row -> row[feature] == x, feature_table)[1,end]
    catch
        val = 0
    end

    # apply laplace smoothing
    return (val+1)/(total+1)
end

# run test
function test(x_test)
    predictions = []

    # iterate all training data
    for i in 1:size(x_test)[1]
        test_case = x_test[i,:]
    
        p_satisfied_proportional = 1
        p_not_satisfied_proportional = 1

        # get probabilities of all features
        for col_name in names(test_case)

            # treat discrete and continuous features separately
            if col_name ∈ disc_col_names
                p_satisfied_proportional *= disc_cond_prob(cond_prob_table, col_name, 1, test_case[col_name])
                p_not_satisfied_proportional *= disc_cond_prob(cond_prob_table, col_name, 0, test_case[col_name])
            else
                p_satisfied_proportional *= likelihood(cond_prob_table, col_name, 1, test_case[col_name])
                p_not_satisfied_proportional *= likelihood(cond_prob_table, col_name, 0, test_case[col_name])
            end
        end

        # calculate probabilities
        p_satisfied = (p_satisfied_proportional / (p_satisfied_proportional+p_not_satisfied_proportional))
        p_not_satisfied = (p_not_satisfied_proportional / (p_satisfied_proportional+p_not_satisfied_proportional))
        
        # count correct and incorrect predictions
        if p_satisfied >= 0.5
            push!(predictions, 1) 
        else
            push!(predictions, 0) 
        end
    end

    return predictions
end

test (generic function with 2 methods)

In [244]:
y_pred = test(x_test)

cm = CM.confmat(y_pred, y_test[:,"satisfaction"])
display(cm)
println("Total correct predictions: ", (cm(0,0) + cm(1,1)))
println("Total incorrect predictions: ", (cm(1,0) + cm(0,1)))
println("Total test rows: ", size(y_test)[1])
println("Model accuracy: ",(cm(0,0) + cm(1,1)) / size(y_test)[1])

          ┌─────────────┐
          │Ground Truth │
┌─────────┼──────┬──────┤
│Predicted│ 0.0  │ 1.0  │
├─────────┼──────┼──────┤
│   0.0   │ 2005 │ 279  │
├─────────┼──────┼──────┤
│   1.0   │ 191  │ 2096 │
└─────────┴──────┴──────┘


Total correct predictions: 4101
Total incorrect predictions: 470
Total test rows: 4571
Model accuracy: 0.8971778604244148
